# **P1 - Finding Lane Lines on the Road** 

[//]: # (Image References)

[origin1]: ./writeup/origin_solidWhiteCurve.jpg "solidWhiteCurve_origin"
[canny1]: ./writeup/canny_solidWhiteCurve.jpg "solidWhiteCurve_canny"
[basicHough1]: ./writeup/hough_solidWhiteCurve.jpg "solidWhiteCurve_basic_hough"
[advancedHough1]: ./writeup/advanced_hough_solidWhiteCurve.jpg "solidWhiteCurve_advanced_hough"
[basicOuput1]: ./writeup/output_solidWhiteCurve.jpg "solidWhiteCurve_basic_output"
[advancedOuput1]: ./writeup/advanced_output_solidWhiteCurve.jpg "solidWhiteCurve_advanced_ouput"
[modified1]: ./writeup/modified_solidWhiteCurve.jpg "solidWhiteCurve_advanced_ouput"

[origin2]: ./writeup/origin_solidYellowCurve.jpg "solidWhiteCurve_origin"
[canny2]: ./writeup/origin_solidYellowCurve.jpg "solidWhiteCurve_canny"
[basicHough2]: ./writeup/origin_solidYellowCurve.jpg "solidWhiteCurve_basic_hough"
[advancedHough2]: ./writeup/origin_solidYellowCurve.jpg "solidWhiteCurve_advanced_hough"
[basicOuput2]: ./writeup/origin_solidYellowCurve.jpg "solidWhiteCurve_basic_output"
[advancedOuput2]: ./writeup/origin_solidYellowCurve.jpg "solidWhiteCurve_advanced_ouput"
---

## Reflection

### 1. Describe your pipeline. As part of the description, explain how you modified the draw_lines() function.

My pipeline consisted of 5 steps. 

**1) Converted the images to grayscale**

**2) Applied `gaussian_blur` with a kernel of a size of 5**

**3) Applied `canny` with 140 as the low threshold and 280 as the high threshold**

To avoid more noise when processing images, I prefer to set a larger threshold to do so. But in my tests, 140 for the low threshold and 280 for the high threshold are the really large values for both, because important information losts when they are getting larger. And I think losing too much significant information to cancel noise is not a worthy tradeoff. 

![alt text][origin1]
![alt text][canny1]

**4) Found hough_lines**
```python
# params setting
img = hough_lines(img, rho=1, theta=np.pi/180, \
                    threshold=9, min_line_len=3, max_line_gap=8)
```

`min_line_len` does not change a lot below 20, so I just set it to 3 and avoid some single pixel noise. 

`max_line_gap` can not be set too large for it will connect the lines that are not relevant.

The last but the most important, `threshold` define the threshold of intersection points to judge whether points are on a line. As my tests, setting it over 12 is too large for detecting the 'little points' on the dotted-lines.

![alt text][canny1]
![alt text][basicHough1]

**5) Drew the lines to visualize the result**

First to see the result of both.
![alt text][origin1]
![alt text][basicHough1]
![alt text][advancedHough1]


In order to draw a single line on the left and right lanes, I modified the draw_lines() function and splited it into 3 steps.

i) Extend the lines to intersect with two lines shown below (y1=340, y2=540) and records the axis of the intersecttion points.
![alt text][modified1]

ii) Accumulate the axis of the intersection points within 4 specified area. Sum up the axis of the points where y=540 and x is in \[110, 210\](left bottom) or \[820, 920\](right bottom), or where y=340 and x is in \[400, 460\](left top) or \[490, 550\](right top). And take the means of them as the axis of my detected lane lines.

iii) Draw it on images.
![alt text][advancedHough1]

### 2. Identify potential shortcomings with your current pipeline

There are two shortcomings with my current pipeline.

One potential shortcoming would be what would happen when curve appears, because this pipeline only draw straight lines (as I the code in `draw_lines()`)

Another one is unstability of it. As observed in the videos, the lines jitter a lot. I think the reason is when I extend the lines, errors would be amplified.



### 3. Suggest possible improvements to your pipeline

First of all, I think my pipeline was implemented in a very tricky way, kind of like hard-coding the lane lines. So far it works well, because the test videos are straight lanes.

To improve it, I think I will use regression to do it. First I will select the points detected by `canny` within the **lane lines frequently appearing area** in the image. And split it into left and right segments. Finally apply two regressions respectively to them. 

But for this improvement, I just wonder whether define the **area** manually a good way to do so.